### 第5章 合并
- 声明：本文绝大部分内容摘自datawhale 学习教程

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('joyful-pandas-master/data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


#### 一、append与assign
#### 1. append方法
#### a.利用序列添加行（必须指定name）

In [2]:
df_append = df.loc[:3,['Gender','Height']].copy()
df_append

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167


In [3]:
s = pd.Series({'Gender':'F','Height':188},name='new_row')
df_append.append(s)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188


#### b.用DataFrame添加表

In [4]:
df_temp = pd.DataFrame({'Gender':['F','M'],'Height':[188,176]},index=['new_1','new_2'])
df_append.append(df_temp)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_1,F,188
new_2,M,176


#### 2. assign方法
#### a.该方法主要用于添加列，列名直接由参数指定：

In [5]:
s=pd.Series(list('abcd'),index=range(4))
df_append.assign(Letter=s)

,Gender,Height,Letter
0,M,173,a
1,F,192,b
2,M,186,c
3,F,167,d


#### 可以一次添加多个列：

In [7]:
df_append.assign(col1=lambda x:x['Gender']*2,
                 col2=s)

,Gender,Height,col1,col2
0,M,173,MM,a
1,F,192,FF,b
2,M,186,MM,c
3,F,167,FF,d


#### 二、combine 与update
#### 1. comine方法
- comine和update都是用于表的填充函数，可以根据某种规则填充

#### a.填充对象
- 可以看出combine方法是按照表的顺序轮流进行逐列循环的，而且自动索引对齐，缺失值为NaN，理解这一点很重要

In [8]:
df_combine_1 = df.loc[:1,['Gender','Height']].copy()
df_combine_2 = df.loc[10:11,['Gender','Height']].copy()
df_combine_1.combine(df_combine_2,lambda x,y:print(x,y))

0       M
1       F
10    NaN
11    NaN
Name: Gender, dtype: object 0     NaN
1     NaN
10      M
11      F
Name: Gender, dtype: object
0     173.0
1     192.0
10      NaN
11      NaN
Name: Height, dtype: float64 0       NaN
1       NaN
10    161.0
11    175.0
Name: Height, dtype: float64


,Gender,Height
0,NaN,NaN
1,NaN,NaN
10,NaN,NaN
11,NaN,NaN


#### b.一些例子
- 例1：根据列均值的大小填充

In [9]:
# 例子1
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [8, 7], 'B': [6, 5]})
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B
0,8,6
1,7,5


- 例2：索引对齐特性（默认状态下，后面的表没有的行列都会设置为NaN）

In [10]:
df2 = pd.DataFrame({'B': [8, 7], 'C': [6, 5]},index=[1,2])
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B,C
0,NaN,NaN,NaN
1,NaN,8.0,6.0
2,NaN,7.0,5.0


- 例3：使得df1原来符合条件的值不会被覆盖

In [11]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,overwrite=False) 

,A,B,C
0,1.0,NaN,NaN
1,2.0,8.0,6.0
2,NaN,7.0,5.0


- 例4：在新增匹配df2的元素位置填充-1

In [13]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=-1)

,A,B,C
0,1.0,-1.0,-1.0
1,2.0,8.0,6.0
2,-1.0,7.0,5.0


#### c.combine_first方法 
- 这个方法作用是用df2填补df1的缺失值，功能比较简单，但很多时候会比combine更常用，下面举两个例子：

In [14]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,0.0,4.0


In [15]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
df1.combine_first(df2)

,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


#### 2. update方法
#### a.三个特点
- 1）返回的框索引只会与被调用框的一致（默认使用左连接，下一节会介绍）
- 2）第二个框中的nan元素不会起作用
- 3）没有返回值，直接在df上操作
#### b.例子
- 例1：索引完全对齐情况下的操作

In [16]:
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df1.update(df2)
df1

,A,B
0,1,4
1,2,5
2,3,6


- 例2：部分填充

In [17]:
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'B': ['d', 'e']}, index=[1,2])
df1.update(df2)
df1

,A,B
0,a,x
1,b,d
2,c,e


- 例3：缺失值不会填充

In [18]:
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, np.nan, 6]})
df1.update(df2)
df1

,A,B
0,1,4.0
1,2,500.0
2,3,6.0


#### 二、concat方法
- concat方法可以在两个维度上拼接，默认纵向凭借（axis=0），拼接方式默认外连接
- 所谓外连接，就是取拼接方向的并集，而'inner'时取拼接方向（若使用默认的纵向拼接，则为列的交集）的交集
- 下面举一些例子说明其参数：

In [19]:
df1 = pd.DataFrame({'A': ['A0', 'A1'],
                    'B': ['B0', 'B1']},
                    index = [0,1])
df2 = pd.DataFrame({'A': ['A2', 'A3'],
                    'B': ['B2', 'B3']},
                    index = [2,3])
df3 = pd.DataFrame({'A': ['A1', 'A3'],
                    'D': ['D1', 'D3'],
                    'E': ['E1', 'E3']},
                    index = [1,3])

- 默认状态拼接：

In [20]:
pd.concat([df1,df2])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


- axis=1时沿列方向拼接：

In [22]:
pd.concat([df1,df2],axis=1)

,A,B,A,B
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,A2,B2
3,NaN,NaN,A3,B3


- join设置为内连接（由于axis=0，因此列取交集）：

In [23]:
pd.concat([df3,df1],join='inner')

,A
1,A1
3,A3
0,A0
1,A1


- join设置为外链接：

In [24]:
pd.concat([df3,df1],join='outer',sort=True) #sort设置列排序，默认为False

,A,B,D,E
1,A1,NaN,D1,E1
3,A3,NaN,D3,E3
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN


- verify_integrity检查列是否唯一：

In [ ]:
#pd.concat([df3,df1],verify_integrity=True,sort=True) 报错

- 同样，可以添加Series：

In [25]:
s = pd.Series(['X0', 'X1'], name='X')
pd.concat([df1,s],axis=1)

,A,B,X
0,A0,B0,X0
1,A1,B1,X1


- key参数用于对不同的数据框增加一个标号，便于索引：

In [26]:
pd.concat([df1,df2], keys=['x', 'y'])
pd.concat([df1,df2], keys=['x', 'y']).index

MultiIndex([('x', 0),
            ('x', 1),
            ('y', 2),
            ('y', 3)],
           )

#### 四、merge与join
#### 1. merge函数
- merge函数的作用是将两个pandas对象横向合并，遇到重复的索引项时会使用笛卡尔积，默认inner连接，可选left、outer、right连接
- 所谓左连接，就是指以第一个表索引为基准，右边的表中如果不再左边的则不加入，如果在左边的就以笛卡尔积的方式加入
- merge/join与concat的不同之处在于on参数，可以指定某一个对象为key来进行连接
- 同样的，下面举一些例子：

In [28]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']}) 
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3']})

- 以key1为准则连接，如果具有相同的列，则默认suffixes=('_x','_y')：

In [29]:
pd.merge(left, right, on='key1')

,key1,key2_x,A,B,key2_y,C,D
0,K0,K0,A0,B0,K0,C0,D0
1,K0,K1,A1,B1,K0,C0,D0
2,K1,K0,A2,B2,K0,C1,D1
3,K1,K0,A2,B2,K0,C2,D2
4,K2,K1,A3,B3,K0,C3,D3


- 以多组键连接：

In [30]:
pd.merge(left, right, on=['key1','key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


- 默认使用inner连接，因为merge只能横向拼接，所以取行向上keys的交集，下面看如果使用how=outer参数
- 注意：这里的how就是concat的join

In [31]:
pd.merge(left, right, how='outer', on=['key1','key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


- 左连接：

In [32]:
pd.merge(left, right, how='left', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


- 右连接：

In [33]:
pd.merge(left, right, how='right', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


- 如果还是对笛卡尔积不太了解，请务必理解下面这个例子，由于B的所有元素为2，因此需要6行：

In [34]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 2, 2]})
pd.merge(left, right, on='B', how='outer')

,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


- validate检验的是到底哪一边出现了重复索引，如果是“one_to_one”则两侧索引都是唯一，如果"one_to_many"则左侧唯一

In [35]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 3, 4]})
#pd.merge(left, right, on='B', how='outer',validate='one_to_one') #报错

In [36]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 1]})
pd.merge(left, right, on='B', how='outer',validate='one_to_one')

,A_x,B,A_y
0,1.0,2,4.0
1,2.0,1,NaN
2,NaN,3,5.0
3,NaN,4,6.0


- indicator参数指示了，合并后该行索引的来源

In [38]:
df1 = pd.DataFrame({'col1': [0, 1], 'col_left': ['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2], 'col_right': [2, 2, 2]})
pd.merge(df1, df2, on='col1', how='outer', indicator=True) #indicator='indicator_column'也是可以的

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


#### 2. join函数
- join函数作用是将多个pandas对象横向拼接，遇到重复的索引项时会使用笛卡尔积，默认左连接，可选inner、outer、right连接

In [39]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                    index=['K0', 'K2', 'K3'])
left.join(right)

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


- 对于many_to_one模式下的合并，往往join更为方便
- 同样可以指定key：

In [40]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                     index=['K0', 'K1'])
left.join(right, on='key')

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


- 多层key：

In [41]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1']})
index = pd.MultiIndex.from_tuples([('K0', 'K0'), ('K1', 'K0'),
                                   ('K2', 'K0'), ('K2', 'K1')],names=['key1','key2'])
right = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']},
                     index=index)
left.join(right, on=['key1','key2'])

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


#### 五、问题与练习
#### 1. 问题

In [ ]:
【问题一】 请思考什么是append/assign/combine/update/concat/merge/join各自最适合使用的场景，并举出相应的例子。

- append/assign 可以用于添加行和列
- comine和update都是用于表的填充函数，可以根据某种规则填充
- concat在行或列两个方向拼接，默认纵向（axis=0），默认外连接（拼接方向的并集）
- merge将两个pandas对象横向合并，遇到重复的index时会使用笛卡尔积，默认inner连接，可选left、outer、right连接。
- join函数作用是将多个pandas对象横向拼接，遇到重复的索引项时会使用笛卡尔积，默认左连接，可选inner、outer、right连接。

In [ ]:
# append 例子，添加行
aList = [123, 'xyz', 'zara', 'abc'];
aList.append( 2009 );
print "Updated List : ", aList

In [ ]:
# assign 函数，新列名为 Letter
s = pd.Series(list('abcd'),index=range(4))
df_append.assign(Letter=s)

In [54]:
#combine 函数的例子
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df=df1.combine_first(df2)

In [48]:
#update 函数例子
# 索引完全对齐的情况。df2 的 B 覆盖 df1 的 B
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df1.update(df2)

In [ ]:
#contact 函数例子
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
ss=pd.concat([s1, s2, s3])

In [52]:
#merge函数例子
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
lr=pd.merge(left1, right1, left_on='key', right_index=True)

In [ ]:
#join 函数的例子
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
lr2=left2.join(right2, how='outer')

In [ ]:
【问题二】 merge_ordered和merge_asof的作用是什么？和merge是什么关系？

- merge_ordered：拼接类似时间序列数据等的有序数据
- merge_asof:类似于有序的left-join ，除了匹配最近的键而不是相等的键之外

【问题三】 请构造一个多级索引与多级索引合并的例子，尝试使用不同的合并函数。

In [ ]:
df1 = pd.DataFrame(np.random.randint(80, 120, size=(2, 4)),
                   index= ['girl', 'boy'],
                   columns=[['English', 'English', 'Chinese', 'Chinese'],
                         ['like', 'dislike', 'like', 'dislike']])

print(df1)    # 创建多级 列 索引

df1 = pd.DataFrame(np.random.randint(90, 110, size=(3, 6)),
                   index= ['girl', 'boy'],
                   columns=[['dislike', 'Chinese', 'dislike', 'dislike'],
                         ['like', 'disChineselike', 'Chinese', 'dislike']])

print(df1) 

df=df1.combine_first(df2)
print(df) 

In [ ]:
【问题四】 上文提到了连接的笛卡尔积，那么当连接方式变化时（inner/outer/left/right），这种笛卡尔积规则会相应变化吗？请构造相应例子。

- inner:保留两对象都拥有的笛卡尔积
- outer:保留全部的笛卡尔积
- left:保留left对象所拥有的全部的笛卡尔积（默认缺失值用nan填充）
- rifht：保留right对象所拥有的全部的笛卡尔积（默认缺失值用nan填充）
#### 不会发生相应变化

In [59]:
left = pd.DataFrame({'key1': ['K0',  'K1'],
                     'key2': ['K1', 'K0'],
                      'A': ['A1', 'A2'],
                      'B': ['B1', 'B2']}) 
right = pd.DataFrame({'key1': ['K1', 'K2'],
                      'key2': [ 'K0', 'K0'],
                      'C': [ 'C2', 'C3'],
                      'D': [ 'D2', 'D3']})

print(left, '\n')
print(right, '\n')

  key1 key2   A   B
0   K0   K1  A1  B1
1   K1   K0  A2  B2 

  key1 key2   C   D
0   K1   K0  C2  D2
1   K2   K0  C3  D3 



#### 2. 练习

In [ ]:
【练习一】有2张公司的员工信息表，每个公司共有16名员工，共有五个公司，请解决如下问题：

In [60]:
pd.read_csv('joyful-pandas-master/data/Employee1.csv').head()

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,21983
2,A,a4,43,158,62.5,21755
3,A,a6,42,182,76.9,17354
4,A,a7,49,171,94.6,6177


In [63]:
pd.read_csv('joyful-pandas-master/data/Employee2.csv').head()

,Company,Name,Age,Height,Weight,Salary
0,A,a1,30,156,91.2,28133
1,A,a2,50,190,83.4,6673
2,A,a3,34,168,96.6,16503
3,A,a5,51,176,97.2,23294
4,A,a6,37,183,93.2,19256


#### a.每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。

In [80]:
#方法一 
df1 = pd.read_csv('joyful-pandas-master/data/Employee1.csv')
df2 = pd.read_csv('joyful-pandas-master/data/Employee2.csv')

Name = []
for name in df1['Name']:
    if name in list(df2['Name']):
        Name.append(name)
Name

['a1',
 'a3',
 'a6',
 'b1',
 'b3',
 'b7',
 'b15',
 'c3',
 'c10',
 'c12',
 'c13',
 'd5',
 'd10',
 'e8',
 'e10',
 'e11']

In [79]:
#方法一  
df1 = pd.read_csv('joyful-pandas-master/data/Employee1.csv')
df2 = pd.read_csv('joyful-pandas-master/data/Employee2.csv')
M = list(set(df1['Name']).intersection(set(df2['Name'])))
M

['d10',
 'c12',
 'e10',
 'a6',
 'a3',
 'b15',
 'b3',
 'd5',
 'e11',
 'c10',
 'a1',
 'c13',
 'b7',
 'c3',
 'b1',
 'e8']

#### b.将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。

In [76]:
df_b1 = df1[~df1['Name'].isin(L)]
df_b2 = df2[~df2['Name'].isin(L)]
df_b = pd.concat([df_b1,df_b2]).set_index('Name')
df_b.head()

,Company,Age,Height,Weight,Salary
Name,,,,,
a4,A,43,158,62.5,21755
a7,A,49,171,94.6,6177
a8,A,51,168,89.5,3246
a9,A,36,186,62.8,3569
a13,A,58,190,75.9,21854


In [77]:
df_b.shape

(45, 5)

#### c.现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，最后对于没有信息的员工，利用缺失值填充。

In [ ]:
#### 不会，记个答案，方便复习
df2 = pd.read_csv('data/Employee2.csv')
df1['重复'] = ['Y_1' if df1.loc[i,'Name'] in L else 'N' for i in range(df1.shape[0])]
df2['重复'] = ['Y_2' if df2.loc[i,'Name'] in L else 'N' for i in range(df2.shape[0])]
df1 = df1.set_index(['Name','重复'])
df2 = df2.set_index(['Name','重复'])
df_c = pd.concat([df1,df2])
result = pd.DataFrame({'Company':[],'Name':[],'Age':[],'Height':[],'Weight':[],'Salary':[]})
group = df_c.groupby(['Company','重复'])
for i in L:
    first = group.get_group((i[0].upper(),'Y_1')).reset_index(level=1).loc[i,:][-4:]
    second = group.get_group((i[0].upper(),'Y_2')).reset_index(level=1).loc[i,:][-4:]
    mean = group.get_group((i[0].upper(),'N')).reset_index(level=1).mean()
    final = [i[0].upper(),i]
    for j in range(4):
        final.append(first[j] if abs(first[j]-mean[j])<abs(second[j]-mean[j]) else second[j])
    result = pd.concat([result,pd.DataFrame({result.columns.tolist()[k]:[final[k]] for k in range(6)})])
result = pd.concat([result.set_index('Name'),df_b])
for i in list('abcde'):
    for j in range(1,17):
        item = i+str(j)
        if item not in result.index:
            result = pd.concat([result,pd.DataFrame({'Company':[i.upper()],'Name':[item]
                 ,'Age':[np.nan],'Height':[np.nan],'Weight':[np.nan],'Salary':[np.nan]}).set_index('Name')])
result['Number'] = [int(i[1:]) for i in result.index]
result.reset_index().drop(columns='Name').set_index(['Company','Number']).sort_index()

【练习二】有2张课程的分数表（分数随机生成），但专业课（学科基础课、专业必修课、专业选修课）与其他课程混在一起，请解决如下问题：

In [65]:
pd.read_csv('joyful-pandas-master/data/Course1.csv').head()

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0


In [66]:
pd.read_csv('joyful-pandas-master/data/Course2.csv').head()

,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0


#### a.将两张表分别拆分为专业课与非专业课（结果为四张表）。

In [85]:
###方法一
df1 = pd.read_csv('joyful-pandas-master/data/Course1.csv')
df2 = pd.read_csv('joyful-pandas-master/data/Course2.csv')

a = ["学科基础课","专业必修课","专业选修课"]
df_a11 = df1[df1["课程类别"].isin(a)]
df_a12 = df1[~df1["课程类别"].isin(a)]
df_a21 = df2[df2["课程类别"].isin(a)]
df_a22 = df2[~df2["课程类别"].isin(a)]

In [86]:
df_a11

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0
7,开源软件设计与开发,专业选修课,3,81.0
9,大数据应用案例,专业必修课,3,91.0
11,电子商务,专业选修课,3,74.0
12,高等数学（二）,学科基础课,4,92.0
14,计算机视觉,专业选修课,3,NaN


In [87]:
df_a12

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
18,足球（初）,体育类,1,100.0
20,毛泽东思想和中国特色社会主义理论体系概论（一）,思政类,3,99.0
23,学术英语听说（一）,英语类,2,85.0
24,程序正义与经典案例解析,精品课程,2,76.0
25,环境可持续发展,公共任意选修类,2,76.0


In [88]:
df_a21

,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0
5,数据科学与工程算法,专业必修课,3,99.0
6,操作系统,专业必修课,4,89.0
7,分布式模型与编程,专业必修课,4,81.0
8,数据管理系统,专业必修课,6,70.0
9,统计与数据分析基础,专业必修课,4,82.0


In [89]:
df_a22

,课程名字,课程类别,学分,分数
25,学术英语听说（二）,英语类,2,92.0
26,学术英语阅读,英语类,2,72.0
27,学术英语写作,英语类,2,98.0
28,美国社会与文化,英语类,2,77.0
29,马克思主义基本原理概论,思政类,3,95.0
30,军事理论,思政类,2,87.0
31,大学语文,文化传承类,2,100.0
32,现代交通地理学,公共任意选修类,2,88.0
33,植物王国,公共任意选修类,2,92.0


In [92]:
### 方法二
df1 = pd.read_csv('joyful-pandas-master/data/Course1.csv')
df2 = pd.read_csv('joyful-pandas-master/data/Course2.csv')

df1_1 = df1.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
df1_2 = df1.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
df2_1 = df2.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
df2_2 = df2.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')

display(df1_1.head(10))
display(df1_2.head(10))
display(df2_1.head(10))
display(df2_2.head(10))

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0
7,开源软件设计与开发,专业选修课,3,81.0
9,大数据应用案例,专业必修课,3,91.0
11,电子商务,专业选修课,3,74.0
12,高等数学（二）,学科基础课,4,92.0
14,计算机视觉,专业选修课,3,NaN


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
18,足球（初）,体育类,1,100.0
20,毛泽东思想和中国特色社会主义理论体系概论（一）,思政类,3,99.0
23,学术英语听说（一）,英语类,2,85.0
24,程序正义与经典案例解析,精品课程,2,76.0
25,环境可持续发展,公共任意选修类,2,76.0


,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0
5,数据科学与工程算法,专业必修课,3,99.0
6,操作系统,专业必修课,4,89.0
7,分布式模型与编程,专业必修课,4,81.0
8,数据管理系统,专业必修课,6,70.0
9,统计与数据分析基础,专业必修课,4,82.0


,课程名字,课程类别,学分,分数
25,学术英语听说（二）,英语类,2,92.0
26,学术英语阅读,英语类,2,72.0
27,学术英语写作,英语类,2,98.0
28,美国社会与文化,英语类,2,77.0
29,马克思主义基本原理概论,思政类,3,95.0
30,军事理论,思政类,2,87.0
31,大学语文,文化传承类,2,100.0
32,现代交通地理学,公共任意选修类,2,88.0
33,植物王国,公共任意选修类,2,92.0


#### b.将两张专业课的分数表和两张非专业课的分数表分别合并。 

In [93]:
df1 = pd.concat([df1_1, df2_1])
df2 = pd.concat([df1_2, df2_2])
df1
df2

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
18,足球（初）,体育类,1,100.0
20,毛泽东思想和中国特色社会主义理论体系概论（一）,思政类,3,99.0
23,学术英语听说（一）,英语类,2,85.0
24,程序正义与经典案例解析,精品课程,2,76.0
25,环境可持续发展,公共任意选修类,2,76.0


#### c.不使用(a)中的步骤，请直接读取两张表合并后拆分。

In [95]:
df = pd.concat([df1,df2])
df11= df.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
df22= df.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
df11

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
18,足球（初）,体育类,1,100.0
20,毛泽东思想和中国特色社会主义理论体系概论（一）,思政类,3,99.0
23,学术英语听说（一）,英语类,2,85.0
24,程序正义与经典案例解析,精品课程,2,76.0
25,环境可持续发展,公共任意选修类,2,76.0


In [96]:
df22

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
18,足球（初）,体育类,1,100.0
20,毛泽东思想和中国特色社会主义理论体系概论（一）,思政类,3,99.0
23,学术英语听说（一）,英语类,2,85.0
24,程序正义与经典案例解析,精品课程,2,76.0
25,环境可持续发展,公共任意选修类,2,76.0


#### d.专业课程中有缺失值吗，如果有的话请在完成(3)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。

In [97]:
df_11 = pd.concat([df1,df2])
df_11['分数'] = df_11.groupby('课程类别').transform(lambda x: x.fillna(x.mean()))['分数']
df_111 = df_11[df_11['课程类别'].isin(["学科基础课","专业必修课","专业选修课"])]
df_222 = df_11[~df_11['课程类别'].isin(["学科基础课","专业必修课","专业选修课"])]

In [99]:
df_111.head(10)

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.000000
2,社会计算,专业选修课,3,78.000000
3,深度学习,专业选修课,3,75.000000
4,人工智能导论,专业必修课,3,84.000000
6,数据结构与算法,学科基础课,5,82.000000
7,开源软件设计与开发,专业选修课,3,81.000000
9,大数据应用案例,专业必修课,3,91.000000
11,电子商务,专业选修课,3,74.000000
12,高等数学（二）,学科基础课,4,92.000000
14,计算机视觉,专业选修课,3,85.791667


In [100]:
df_222.head(10)

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
18,足球（初）,体育类,1,100.0
20,毛泽东思想和中国特色社会主义理论体系概论（一）,思政类,3,99.0
23,学术英语听说（一）,英语类,2,85.0
24,程序正义与经典案例解析,精品课程,2,76.0
25,环境可持续发展,公共任意选修类,2,76.0


#### 学习小结

- 今天的课程是本次学习最后一次作业了，很开心能参与这次学习，虽然时间不长，还是学到了很多东西，最近也是因为这次学习让我每天回家都有事情做，感觉特别充实。自己学习东一点、西一点，学一点、忘一点，形不成比较完整的知识体系，能跟着教程完整的学一遍，真的收获良多，我特别喜欢完成任务的感觉，每次打卡总感觉自己又干好了一件事，很有满足感。而且每次学习都有打卡笔记，不管当时真正学懂多少，消化多少，以后翻出来再看都会更有所悟、更有所得，所以感谢datawhale 做了这么棒的开源学习教程，还有这么好的学习方式。祝组织越办越好，也希望以后还能有机会参与更多的学习，自己可以越来越优秀，每天进步一点点，每天变好一点点，愿世界充满爱，每个人都是四季如风宛如清泉清澈的少年。。最后，我还想学，下次希望还能报名成功，自己学真的容易坚持不下去，有人一起就不容易放弃，有了时间紧迫感，也能做事更专注！加油！